In [ ]:
!pip install --upgrade mxnet
!pip install autogluon

In [ ]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.device_count())  # Should be > 0

True
1


In [ ]:
import autogluon.core as ag
from autogluon.vision import ImagePredictor, ImageDataset

/usr/local/lib/python3.7/dist-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.0` and `torch==1.10.0+cu111` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/house-grade/
!mkdir /content/properties-tagging/
# !mkdir /content/thai-food-labeling/

!unzip /content/drive/MyDrive/super-ai-engineer-2021-house-grade-classification.zip -d /content/house-grade/
!unzip /content/drive/MyDrive/super-ai-engineer-2021-properties-tagging.zip -d /content/properties-tagging/
# !unzip /content/super-ai-engineer-2021-thai-food-labeling.zip -d /content/thai-food-labeling/

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
  inflating: /content/properties-tagging/extra/img_13-7376641365172_100-561545971402_s90_f90_y0_1_09-2020.jpg  
  inflating: /content/properties-tagging/extra/img_13-7377315062526_100-559460968232_s270_f90_y0_1_03-2021.jpg  
  inflating: /content/properties-tagging/extra/img_13-7377315062526_100-559460968232_s90_f90_y0_1_03-2021.jpg  
  inflating: /content/properties-tagging/extra/img_13-7377530416811_100-561562662245_s270_f90_y0_1_09-2020.jpg  
  inflating: /content/properties-tagging/extra/img_13-7377530416811_100-561562662245_s90_f90_y0_1_09-2020.jpg  
  inflating: /content/properties-tagging/extra/img_13-7377896366311_100-561453030533_s270_f90_y0_1_09-2020.jpg  
  inflating: /content/properties-tagging/extra/img_13-7377896366311_100-561453030533_s90_f90_y0_1_09-2020.jpg  
  inflating: /content/properties-tagging/extra/img_13-7378416613642_100-561580866435_s270_f90_y0_1_09-2020.jpg  
  inflating: /content/properties-tagging/ex

In [ ]:
!unzip /content/drive/MyDrive/archive_3.zip -d /content

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
  inflating: /content/House_price_data/train/img_13-779683_100-5642368_a185-6541727_s95-6541727_y0_f90_1.jpg  
  inflating: /content/House_price_data/train/img_13-7798511_100-6253071_a173-7864608_s263-7864608_y0_f90_1.jpg  
  inflating: /content/House_price_data/train/img_13-7798511_100-6253071_a173-7864608_s83-78646079999999_y0_f90_1.jpg  
  inflating: /content/House_price_data/train/img_13-7798976_100-6279134_a168-8064331_s258-8064331_y0_f90_1.jpg  
  inflating: /content/House_price_data/train/img_13-7799194_100-6106338_a3-910038147_s273-910038147_y0_f90_1.jpg  
  inflating: /content/House_price_data/train/img_13-7799194_100-6106338_a3-910038147_s93-910038147_y0_f90_1.jpg  
  inflating: /content/House_price_data/train/img_13-7799421_100-580287_a116-9021457_s206-9021457_y0_f90_1.jpg  
  inflating: /content/House_price_data/train/img_13-7799421_100-580287_a116-9021457_s26-902145700000005_y0_f90_1.jpg  
  inflating: /content/House

In [ ]:
import pandas as pd

In [ ]:
train_df=pd.read_csv('/content/drive/MyDrive/manual_label.csv')
train_df=train_df.reset_index()
train_df['image']=train_df['filename'].apply(lambda x: '/content/house-grade/train/'+x)
# train_df.drop(['filename','index'],axis=1,inplace=True)
# train_df.set_index(['image'],inplace=True)
train_df.head()

,index,filename,labels,image
0,0,ChokChai4_img_13-7955799_100-6031141_a187-2159...,0,/content/house-grade/train/ChokChai4_img_13-79...
1,1,ChokChai4_img_13-7956791_100-6031267_a187-2159...,0,/content/house-grade/train/ChokChai4_img_13-79...
2,2,ChokChai4_img_13-7956791_100-6031267_a187-2159...,5,/content/house-grade/train/ChokChai4_img_13-79...
3,3,ChokChai4_img_13-7961753_100-6031881_a185-9785...,0,/content/house-grade/train/ChokChai4_img_13-79...
4,4,ChokChai4_img_13-7969811_100-5906061_a180-5812...,0,/content/house-grade/train/ChokChai4_img_13-79...


In [ ]:
augment_df=pd.read_csv('/content/House_price_data/data_augment_1_4.csv')
augment_df.head()

,filename,labels
0,House_price_data/augment/aug_label1_id18_angle...,1 3
1,House_price_data/augment/aug_label1_id18_angle...,1 3
2,House_price_data/augment/aug_label1_id18_angle...,1 3
3,House_price_data/augment/aug_label1_id18_angle...,1 3
4,House_price_data/augment/aug_label1_id18_angle...,1 3


In [ ]:
augment_df['filename']=augment_df['filename'].str.split('/').str[-1]
augment_df

,filename,labels
0,aug_label1_id18_angle0_flip.jpg,1 3
1,aug_label1_id18_angle1_flip.jpg,1 3
2,aug_label1_id18_angle2_flip.jpg,1 3
3,aug_label1_id18_angle1.jpg,1 3
4,aug_label1_id18_angle2.jpg,1 3
...,...,...
1565,aug_label4_id5872_angle0_flip.jpg,4 5
1566,aug_label4_id5872_angle1_flip.jpg,4 5
1567,aug_label4_id5872_angle2_flip.jpg,4 5
1568,aug_label4_id5872_angle1.jpg,4 5


In [ ]:
augment_df['image']=augment_df['filename'].apply(lambda x: '/content/House_price_data/train/'+x)
augment_df.head()

,filename,labels,image
0,aug_label1_id18_angle0_flip.jpg,1 3,/content/House_price_data/train/aug_label1_id1...
1,aug_label1_id18_angle1_flip.jpg,1 3,/content/House_price_data/train/aug_label1_id1...
2,aug_label1_id18_angle2_flip.jpg,1 3,/content/House_price_data/train/aug_label1_id1...
3,aug_label1_id18_angle1.jpg,1 3,/content/House_price_data/train/aug_label1_id1...
4,aug_label1_id18_angle2.jpg,1 3,/content/House_price_data/train/aug_label1_id1...


In [ ]:
new_train_df=pd.concat([train_df,augment_df],axis=0)
new_train_df

,index,filename,labels,image
0,0.0,ChokChai4_img_13-7955799_100-6031141_a187-2159...,0,/content/house-grade/train/ChokChai4_img_13-79...
1,1.0,ChokChai4_img_13-7956791_100-6031267_a187-2159...,0,/content/house-grade/train/ChokChai4_img_13-79...
2,2.0,ChokChai4_img_13-7956791_100-6031267_a187-2159...,5,/content/house-grade/train/ChokChai4_img_13-79...
3,3.0,ChokChai4_img_13-7961753_100-6031881_a185-9785...,0,/content/house-grade/train/ChokChai4_img_13-79...
4,4.0,ChokChai4_img_13-7969811_100-5906061_a180-5812...,0,/content/house-grade/train/ChokChai4_img_13-79...
...,...,...,...,...
1565,NaN,aug_label4_id5872_angle0_flip.jpg,4 5,/content/House_price_data/train/aug_label4_id5...
1566,NaN,aug_label4_id5872_angle1_flip.jpg,4 5,/content/House_price_data/train/aug_label4_id5...
1567,NaN,aug_label4_id5872_angle2_flip.jpg,4 5,/content/House_price_data/train/aug_label4_id5...
1568,NaN,aug_label4_id5872_angle1.jpg,4 5,/content/House_price_data/train/aug_label4_id5...


In [ ]:
new_train_df=new_train_df.drop(['index','filename'],axis=1).set_index('image')
new_train_df

,labels
image,
/content/house-grade/train/ChokChai4_img_13-7955799_100-6031141_a187-21598902886774_s270_y0_f90_1.jpg,0
/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s270_y0_f90_1.jpg,0
/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s90_y0_f90_1.jpg,5
/content/house-grade/train/ChokChai4_img_13-7961753_100-6031881_a185-978546002526_s90_y0_f90_1.jpg,0
/content/house-grade/train/ChokChai4_img_13-7969811_100-5906061_a180-5812875399312_s270_y0_f90_1.jpg,0
...,...
/content/House_price_data/train/aug_label4_id5872_angle0_flip.jpg,4 5
/content/House_price_data/train/aug_label4_id5872_angle1_flip.jpg,4 5
/content/House_price_data/train/aug_label4_id5872_angle2_flip.jpg,4 5


In [ ]:
!pip install --upgrade --no-cache-dir gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=4109cccd52a8d83575be541b2f26d04676f0d95febe0e8de181c86cb1376e821
  Stored in directory: /tmp/pip-ephem-wheel-cache-8i8bmf_o/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [ ]:
!gdown --id 1-y_n2_6rQL-0hIVw1EIDtiQK9J2QZZPk

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-y_n2_6rQL-0hIVw1EIDtiQK9J2QZZPk
To: /content/online_data_no_bandpass.npy
  6% 303M/5.16G [00:02<00:38, 127MB/s]Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/gdown/cli.py", line 166, in main
    resume=args.continue_,
  File "/usr/local/lib/python3.7/dist-packages/gdown/download.py", line 257, in download
    for chunk in res.iter_content(chunk_size=CHUNK_SIZE):
  File "/usr/local/lib/python3.7/dist-packages/requests/models.py", line 751, in generate
    for chunk in self.raw.stream(chunk_size, decode_content=True):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/response.py", line 496, in stream
    

In [ ]:
# !gdown --id 1090jAQw-fqeE5BiejtFXQ-6X0JPPtW8q

In [ ]:
# !unzip /content/GoogleStreetView-ศาลพระภูมิ-V2.zip -d bi_sanprapoom

In [ ]:
# from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
# from PIL import Image
# import requests

# feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b5-finetuned-ade-640-640")
# model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b5-finetuned-ade-640-640")

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

# inputs = feature_extractor(images=image, return_tensors="pt")
# outputs = model(**inputs)
# logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)

In [ ]:
# outputs[0].shape

In [ ]:
# outputs[0][0]

In [ ]:
# import torch
# torch.argmax(outputs[0][0],dim=0)

In [ ]:
new_train_df.index

Index(['/content/house-grade/train/ChokChai4_img_13-7955799_100-6031141_a187-21598902886774_s270_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s270_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s90_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7961753_100-6031881_a185-978546002526_s90_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7969811_100-5906061_a180-5812875399312_s270_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7970811_100-5906071_a180-5812875399312_s90_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7971811_100-5906081_a180-5812875399312_s90_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7972811_100-5906092_a180-5812875399312_s90_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7973811_100-5906102_a180-5812875399312_s270_y0_f90_1.jpg',
    

In [ ]:
label1=[]
label2=[]
label3=[]
label4=[]
label5=[]

for labels in new_train_df['labels'].str.split(' '):

  if '1' in labels:
    label1.append(1)
  else:
    label1.append(0)
  
  if '2' in labels:
    label2.append(1)
  else:
    label2.append(0)
  
  if '3' in labels:
    label3.append(1)
  else:
    label3.append(0)
  
  if '4' in labels:
    label4.append(1)
  else:
    label4.append(0)
  
  if '5' in labels:
    label5.append(1)
  else:
    label5.append(0)

In [ ]:
new_train_df['label1']=label1
new_train_df['label2']=label2
new_train_df['label3']=label3
new_train_df['label4']=label4
new_train_df['label5']=label5

In [ ]:
new_train_df=new_train_df.drop('labels',axis=1)
new_train_df

,label1,label2,label3,label4,label5
image,,,,,
/content/house-grade/train/ChokChai4_img_13-7955799_100-6031141_a187-21598902886774_s270_y0_f90_1.jpg,0,0,0,0,0
/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s270_y0_f90_1.jpg,0,0,0,0,0
/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s90_y0_f90_1.jpg,0,0,0,0,1
/content/house-grade/train/ChokChai4_img_13-7961753_100-6031881_a185-978546002526_s90_y0_f90_1.jpg,0,0,0,0,0
/content/house-grade/train/ChokChai4_img_13-7969811_100-5906061_a180-5812875399312_s270_y0_f90_1.jpg,0,0,0,0,0
...,...,...,...,...,...
/content/House_price_data/train/aug_label4_id5872_angle0_flip.jpg,0,0,0,1,1
/content/House_price_data/train/aug_label4_id5872_angle1_flip.jpg,0,0,0,1,1
/content/House_price_data/train/aug_label4_id5872_angle2_flip.jpg,0,0,0,1,1


In [ ]:
print(new_train_df['label1'].value_counts())
print(new_train_df['label2'].value_counts())
print(new_train_df['label3'].value_counts())
print(new_train_df['label4'].value_counts())
print(new_train_df['label5'].value_counts())

0    4100
1     423
Name: label1, dtype: int64
0    3354
1    1169
Name: label2, dtype: int64
0    2787
1    1736
Name: label3, dtype: int64
0    3259
1    1264
Name: label4, dtype: int64
0    3759
1     764
Name: label5, dtype: int64


In [ ]:
train_one=pd.DataFrame(new_train_df.loc[:,'label1']).rename(columns={'label1':'label'}).reset_index()
train_two=pd.DataFrame(new_train_df.loc[:,'label2']).rename(columns={'label2':'label'}).reset_index()
train_three=pd.DataFrame(new_train_df.loc[:,'label3']).rename(columns={'label3':'label'}).reset_index()
train_four=pd.DataFrame(new_train_df.loc[:,'label4']).rename(columns={'label4':'label'}).reset_index()
train_five=pd.DataFrame(new_train_df.loc[:,'label5']).rename(columns={'label5':'label'}).reset_index()

In [ ]:
train_three

,image,label
0,/content/house-grade/train/ChokChai4_img_13-79...,0
1,/content/house-grade/train/ChokChai4_img_13-79...,0
2,/content/house-grade/train/ChokChai4_img_13-79...,0
3,/content/house-grade/train/ChokChai4_img_13-79...,0
4,/content/house-grade/train/ChokChai4_img_13-79...,0
...,...,...
4518,/content/House_price_data/train/aug_label4_id5...,0
4519,/content/House_price_data/train/aug_label4_id5...,0
4520,/content/House_price_data/train/aug_label4_id5...,0
4521,/content/House_price_data/train/aug_label4_id5...,0


In [ ]:
import cv2
import numpy as np
from tqdm import tqdm

In [ ]:
# for image_dir in tqdm(train_three['image'][2187+470:]):
#   image = Image.open(image_dir)
#   image = np.asarray(image, dtype=np.float32) / 255
#   image = image[:, :, :3]

#   inputs = feature_extractor(images=image, return_tensors="pt")
#   outputs = model(**inputs)
#   logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)

#   image_label=torch.argmax(logits[0],dim=0).numpy().astype(np.uint8)
#   cv2.imwrite('/content/drive/MyDrive/segment/'+image_dir.split('/')[-1], image_label)

In [ ]:
import random

In [ ]:
# train_one_drop=random.sample(list(train_one.loc[train_one.label==0,:].index),3600)
# train_two_drop=random.sample(list(train_two.loc[train_two.label==0,:].index),2000)
# train_three_drop=random.sample(list(train_three.loc[train_three.label==0,:].index),1000)
# train_four_drop=random.sample(list(train_four.loc[train_four.label==0,:].index),2000)
# train_five_drop=random.sample(list(train_five.loc[train_five.label==0,:].index),3000)

In [ ]:
# train_one=train_one.drop(train_one_drop)
# train_two=train_two.drop(train_two_drop)
# train_three=train_three.drop(train_three_drop)
# train_four=train_four.drop(train_four_drop)
# train_five=train_five.drop(train_five_drop)

In [ ]:
# print(train_one['label'].value_counts())
# print(train_two['label'].value_counts())
# print(train_three['label'].value_counts())
# print(train_four['label'].value_counts())
# print(train_five['label'].value_counts())

In [ ]:
# train_three['image']='/content/drive/MyDrive/segment/'+train_three['image'].str.split('/').str[-1]
# train_three

In [ ]:
# train_five['image']='/content/drive/MyDrive/segment/'+train_five['image'].str.split('/').str[-1]
# train_five

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
from autogluon.core.metrics import make_scorer
ag_f1_scorer = make_scorer(name='f1_score',
                                 score_func=f1_score,
                                 optimum=1,
                                 greater_is_better=True)

In [ ]:
ImagePredictor.list_models()

('resnet18_v1',
 'resnet34_v1',
 'resnet50_v1',
 'resnet101_v1',
 'resnet152_v1',
 'resnet18_v2',
 'resnet34_v2',
 'resnet50_v2',
 'resnet101_v2',
 'resnet152_v2',
 'resnest14',
 'resnest26',
 'resnest50',
 'resnest101',
 'resnest200',
 'resnest269',
 'se_resnet18_v1',
 'se_resnet34_v1',
 'se_resnet50_v1',
 'se_resnet101_v1',
 'se_resnet152_v1',
 'se_resnet18_v2',
 'se_resnet34_v2',
 'se_resnet50_v2',
 'se_resnet101_v2',
 'se_resnet152_v2',
 'vgg11',
 'vgg13',
 'vgg16',
 'vgg19',
 'vgg11_bn',
 'vgg13_bn',
 'vgg16_bn',
 'vgg19_bn',
 'alexnet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'shufflenet_v1',
 'shufflenet_v2',
 'squeezenet1.0',
 'squeezenet1.1',
 'googlenet',
 'inceptionv3',
 'xception',
 'xception71',
 'mobilenet1.0',
 'mobilenet0.75',
 'mobilenet0.5',
 'mobilenet0.25',
 'mobilenetv2_1.0',
 'mobilenetv2_0.75',
 'mobilenetv2_0.5',
 'mobilenetv2_0.25',
 'mobilenetv3_large',
 'mobilenetv3_small',
 'cifar_resnet20_v1',
 'cifar_resnet56_v1',
 'cifar_resnet110

In [ ]:
predictor_one = ImagePredictor(eval_metric=ag_f1_scorer)
predictor_one.fit(train_one, hyperparameters={'epochs': 20, 'batch_size':32, 'model':'efficientnet_b0'},time_limit=60*60)

Converting raw DataFrame to ImageDataset...
Detected 2 unique classes: [0, 1]
If you feel the `classes` is inaccurate, please construct the dataset explicitly, e.g. train_data = ImageDataset(train_data, classes=["foo", "bar"])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Randomly split train_data into train[4070]/validation[453] splits.
The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1
Starting fit without HPO
modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != efficientnet_b0
root.train.early_stop_max_value 1.0 != inf
root.train.early_stop_patience -1 != 10
root.train.epochs    200 != 20
root.train.early_stop_baseline 0.0 != -inf
root.misc.seed       42 != 631
}
Saved config to /content/2a29cfb4/.trial_0/config.yaml
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342d

In [ ]:
predictor_two = ImagePredictor()
predictor_two.fit(train_two, hyperparameters={'epochs': 20, 'batch_size':32, 'model':'efficientnet_b0'},time_limit=60*60)

In [ ]:
predictor_three = ImagePredictor()
predictor_three.fit(train_three, hyperparameters={'epochs': 30, 'batch_size':32, 'model':'efficientnet_b0'},time_limit=60*60)

Converting raw DataFrame to ImageDataset...
Detected 2 unique classes: [0, 1]
If you feel the `classes` is inaccurate, please construct the dataset explicitly, e.g. train_data = ImageDataset(train_data, classes=["foo", "bar"])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Randomly split train_data into train[4070]/validation[453] splits.
The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1
Starting fit without HPO
modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != efficientnet_b0
root.misc.seed       42 != 513
root.train.early_stop_patience -1 != 10
root.train.epochs    200 != 30
root.train.early_stop_baseline 0.0 != -inf
root.train.early_stop_max_value 1.0 != inf
}
Saved config to /content/b1fa151b/.trial_0/config.yaml
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342d

In [ ]:
predictor_four = ImagePredictor()
predictor_four.fit(train_four, hyperparameters={'epochs': 20, 'batch_size':32, 'model':'efficientnet_b0'},time_limit=60*60)

In [ ]:
predictor_five = ImagePredictor()
predictor_five.fit(train_five, hyperparameters={'epochs': 30, 'batch_size':32, 'model':'efficientnet_b0'},time_limit=60*60)

In [ ]:
predictor_one.save('/content/drive/MyDrive/autogluon model/predictor1.ag')
predictor_two.save('/content/drive/MyDrive/autogluon model/predictor2.ag')
predictor_three.save('/content/drive/MyDrive/autogluon model/predictor3.ag')
predictor_four.save('/content/drive/MyDrive/autogluon model/predictor4.ag')
predictor_five.save('/content/drive/MyDrive/autogluon model/predictor5.ag')

In [ ]:
predictor_three.save('/content/drive/MyDrive/autogluon model/predictor3_original.ag')
predictor_five.save('/content/drive/MyDrive/autogluon model/predictor5_original.ag')

In [ ]:
# predict0=predictor_zero.predict(valid_zero)
# predict1=predictor_one.predict(valid_one)
# predict2=predictor_two.predict(valid_two)
# predict3=predictor_three.predict(valid_three)
# predict4=predictor_four.predict(valid_four)
# predict5=predictor_five.predict(valid_five)

In [ ]:
# print(train_df.loc[:,'label0'].value_counts())
# print(train_df.loc[:,'label1'].value_counts())
# print(train_df.loc[:,'label2'].value_counts())
# print(train_df.loc[:,'label3'].value_counts())
# print(train_df.loc[:,'label4'].value_counts())
# print(train_df.loc[:,'label5'].value_counts())

In [ ]:
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import accuracy_score

# print('0')
# print(accuracy_score(validate_df['label0'],predict0))
# print(precision_score(validate_df['label0'],predict0))
# print(recall_score(validate_df['label0'],predict0))
# print('\n')
# print('1')
# print(accuracy_score(validate_df['label1'],predict1))
# print(precision_score(validate_df['label1'],predict1))
# print(recall_score(validate_df['label1'],predict1))
# print('\n')
# print('2')
# print(accuracy_score(validate_df['label2'],predict2))
# print(precision_score(validate_df['label2'],predict2))
# print(recall_score(validate_df['label2'],predict2))
# print('\n')
# print('3')
# print(accuracy_score(validate_df['label3'],predict3))
# print(precision_score(validate_df['label3'],predict3))
# print(recall_score(validate_df['label3'],predict3))
# print('\n')
# print('4')
# print(accuracy_score(validate_df['label4'],predict4))
# print(precision_score(validate_df['label4'],predict4))
# print(recall_score(validate_df['label4'],predict4))
# print('\n')
# print('5')
# print(accuracy_score(validate_df['label5'],predict5))
# print(precision_score(validate_df['label5'],predict5))
# print(recall_score(validate_df['label5'],predict5))

In [ ]:
test_df=pd.read_csv('/content/properties-tagging/sample_submission.csv')
test_df

,Id,Labels
0,d5461e01,2 3
1,0e6bd6ea,0
2,03df66ca,3
3,bdf217f5,NaN
4,632726ba,NaN
...,...,...
1545,98768d3f,NaN
1546,34f08e48,NaN
1547,4337ff85,NaN
1548,825b04fe,NaN


In [ ]:
import os
images=[]
image_dirs=[]
for image in os.listdir('/content/house-grade/test'):
  image_dir='/content/house-grade/test/'+image
  images.append(image[:-4])
  image_dirs.append(image_dir)

In [ ]:
test_df=pd.DataFrame({'image':image_dirs,'Id':images})
test_df

,image,Id
0,/content/house-grade/test/ad6a7967.jpg,ad6a7967
1,/content/house-grade/test/9f72698b.jpg,9f72698b
2,/content/house-grade/test/ea3d1db5.jpg,ea3d1db5
3,/content/house-grade/test/43c5b7da.jpg,43c5b7da
4,/content/house-grade/test/aa9d5db0.jpg,aa9d5db0
...,...,...
445,/content/house-grade/test/9736ac21.jpg,9736ac21
446,/content/house-grade/test/c346950c.jpg,c346950c
447,/content/house-grade/test/034c3c52.jpg,034c3c52
448,/content/house-grade/test/d011d8c4.jpg,d011d8c4


In [ ]:
# !mkdir test_segment

In [ ]:
# for image_dir in tqdm(test_df['image']):
#   image = Image.open(image_dir)
#   image = np.asarray(image, dtype=np.float32) / 255
#   image = image[:, :, :3]

#   inputs = feature_extractor(images=image, return_tensors="pt")
#   outputs = model(**inputs)
#   logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)

#   image_label=torch.argmax(logits[0],dim=0).numpy().astype(np.uint8)
#   cv2.imwrite('/content/test_segment/'+image_dir.split('/')[-1], image_label)

In [ ]:
# '/content/test_segment/'+test_df['image'].str.split('/').str[-1]

In [ ]:
test_segment=pd.DataFrame({'image':'/content/test_segment/'+test_df['image'].str.split('/').str[-1]})
test_segment

,image
0,/content/test_segment/ad6a7967.jpg
1,/content/test_segment/9f72698b.jpg
2,/content/test_segment/ea3d1db5.jpg
3,/content/test_segment/43c5b7da.jpg
4,/content/test_segment/aa9d5db0.jpg
...,...
445,/content/test_segment/9736ac21.jpg
446,/content/test_segment/c346950c.jpg
447,/content/test_segment/034c3c52.jpg
448,/content/test_segment/d011d8c4.jpg


In [ ]:
# predict_test0=predictor_zero.predict(pd.DataFrame(test_df['image']))
predict_test1=predictor_one.predict(pd.DataFrame(test_df['image']))
predict_test2=predictor_two.predict(pd.DataFrame(test_df['image']))
predict_test3=predictor_three.predict(test_segment)
predict_test4=predictor_four.predict(pd.DataFrame(test_df['image']))
predict_test5=predictor_five.predict(test_segment)

In [ ]:
predicts=[]
for index in range(len(test_df)):
  predict_list=[]
  # if predict_test0[index]==1:
  #   predict_list.append('0')
  if predict_test1[index]==1:
    predict_list.append('1')
  if predict_test2[index]==1:
    predict_list.append('2')
  if predict_test3[index]==1:
    predict_list.append('3')
  if predict_test4[index]==1:
    predict_list.append('4')
  if predict_test5[index]==1:
    predict_list.append('5')
  
  if predict_list==[]:
    predict_list=['0']
  
  # if '0' in predict_list and len(predict_list)>1:
  #   predict_list.pop(0)
  
  predict_concat=' '.join(predict_list)
  predicts.append(predict_concat)

In [ ]:
predicts

['2 3',
 '2 4',
 '2',
 '2 3 4',
 '5',
 '2',
 '3',
 '0',
 '0',
 '5',
 '3',
 '0',
 '0',
 '2 3',
 '2',
 '0',
 '0',
 '2 5',
 '0',
 '0',
 '3 4',
 '0',
 '3',
 '4',
 '0',
 '0',
 '2',
 '3',
 '3',
 '2 3',
 '0',
 '2',
 '0',
 '5',
 '2',
 '3',
 '2 3',
 '2',
 '2 3',
 '0',
 '0',
 '3',
 '3',
 '0',
 '0',
 '3',
 '0',
 '3',
 '3',
 '2',
 '3',
 '3',
 '2 3',
 '0',
 '0',
 '3',
 '3',
 '0',
 '3',
 '0',
 '0',
 '0',
 '3',
 '3 5',
 '0',
 '3',
 '2',
 '2',
 '2',
 '3',
 '3',
 '5',
 '0',
 '0',
 '0',
 '3',
 '0',
 '0',
 '2',
 '2',
 '2',
 '0',
 '3',
 '2',
 '2 5',
 '5',
 '0',
 '3',
 '0',
 '3',
 '1 2 5',
 '2',
 '0',
 '2',
 '2',
 '2 3 5',
 '2',
 '2',
 '0',
 '3',
 '2 3 5',
 '5',
 '0',
 '2',
 '0',
 '0',
 '3',
 '0',
 '3',
 '0',
 '0',
 '2 3',
 '0',
 '2',
 '0',
 '2 3',
 '3',
 '0',
 '2',
 '0',
 '5',
 '5',
 '0',
 '3 5',
 '2 3',
 '0',
 '3',
 '3',
 '2 3',
 '0',
 '0',
 '0',
 '3',
 '0',
 '2 3',
 '3',
 '0',
 '2',
 '5',
 '0',
 '2 3 5',
 '3',
 '0',
 '5',
 '0',
 '3',
 '0',
 '3',
 '2 5',
 '3',
 '4 5',
 '0',
 '0',
 '5',
 '2 3',
 '2',
 '0'

In [ ]:
submit_df=pd.read_csv('/content/properties-tagging/sample_submission.csv')
submit_df

,Id,Labels
0,d5461e01,2 3
1,0e6bd6ea,0
2,03df66ca,3
3,bdf217f5,NaN
4,632726ba,NaN
...,...,...
1545,98768d3f,NaN
1546,34f08e48,NaN
1547,4337ff85,NaN
1548,825b04fe,NaN


In [ ]:
test_df['Labels']=predicts
test_df

,image,Id,Labels
0,/content/house-grade/test/ad6a7967.jpg,ad6a7967,2 3
1,/content/house-grade/test/9f72698b.jpg,9f72698b,2 4
2,/content/house-grade/test/ea3d1db5.jpg,ea3d1db5,2
3,/content/house-grade/test/43c5b7da.jpg,43c5b7da,2 3 4
4,/content/house-grade/test/aa9d5db0.jpg,aa9d5db0,5
...,...,...,...
445,/content/house-grade/test/9736ac21.jpg,9736ac21,0
446,/content/house-grade/test/c346950c.jpg,c346950c,2
447,/content/house-grade/test/034c3c52.jpg,034c3c52,0
448,/content/house-grade/test/d011d8c4.jpg,d011d8c4,2 3


In [ ]:
submit_df=submit_df.merge(test_df,how='left',on='Id')
submit_df

,Id,Labels_x,image,Labels_y
0,d5461e01,2 3,/content/house-grade/test/d5461e01.jpg,0
1,0e6bd6ea,0,/content/house-grade/test/0e6bd6ea.jpg,0
2,03df66ca,3,/content/house-grade/test/03df66ca.jpg,3
3,bdf217f5,NaN,/content/house-grade/test/bdf217f5.jpg,2 3 4
4,632726ba,NaN,/content/house-grade/test/632726ba.jpg,3
...,...,...,...,...
1545,98768d3f,NaN,NaN,NaN
1546,34f08e48,NaN,NaN,NaN
1547,4337ff85,NaN,NaN,NaN
1548,825b04fe,NaN,NaN,NaN


In [ ]:
submit_df=submit_df.drop(['Labels_x','image'],axis=1).set_index('Id').rename(columns={'Labels_y':'Labels'})
submit_df

,Labels
Id,
d5461e01,0
0e6bd6ea,0
03df66ca,3
bdf217f5,2 3 4
632726ba,3
...,...
98768d3f,NaN
34f08e48,NaN
4337ff85,NaN


In [ ]:
submit_df.to_csv('Submission.csv')